In [1]:
# Install required packages
!pip install torch transformers pandas tqdm sentence-transformers rouge-score sacremoses sacrebleu bert-score

# Imports
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import BioGptTokenizer, BioGptForCausalLM
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bertscore
import sacrebleu

# Load model and tokenizer
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load data
df = pd.read_csv("/kaggle/input/precription/health prescription data.csv")
df = df[['SUBJECT_ID', 'ROW_ID', 'HADM_ID', 'CATEGORY', 'ADMISSION_TYPE', 'DIAGNOSIS', 'TEXT']].dropna(subset=['TEXT']).reset_index(drop=True)

# Sample subset
sample_df = df.sample(n=30, random_state=42).reset_index(drop=True)

# Clarification function
def clarify_text(text, max_new_tokens=200):
    prompt = f"Summarize clearly this medical report:\n\n{text}\n\nSummary:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024 - max_new_tokens,
        padding="max_length"
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    clarified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clarified_text.replace(prompt, "").strip()

# Generate summaries
clarified_summaries = []
for report in tqdm(sample_df['TEXT'], desc="Clarifying Medical Reports"):
    clarified = clarify_text(report)
    clarified_summaries.append(clarified)

sample_df['clarified_summary'] = clarified_summaries

# Initialize evaluation tools
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

rougeL_scores = []
cosine_scores = []
original_texts = sample_df['TEXT'].tolist()
clarified_texts = sample_df['clarified_summary'].tolist()

# Compute embeddings
original_embeddings = embed_model.encode(original_texts, convert_to_tensor=True)
clarified_embeddings = embed_model.encode(clarified_texts, convert_to_tensor=True)

# Compute ROUGE-L and Cosine Similarity
for orig, clar, orig_emb, clar_emb in tqdm(zip(original_texts, clarified_texts, original_embeddings, clarified_embeddings), total=len(original_texts)):
    rougeL_score = rouge.score(orig, clar)['rougeL'].fmeasure
    rougeL_scores.append(rougeL_score)
    cosine_sim = util.cos_sim(orig_emb, clar_emb).item()
    cosine_scores.append(cosine_sim)

avg_rougeL = np.mean(rougeL_scores)
avg_cosine_similarity = np.mean(cosine_scores)

print(f"\nAverage ROUGE-L: {avg_rougeL:.4f}")
print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")

# BLEU evaluation
bleu = sacrebleu.corpus_bleu(clarified_texts, [original_texts])
bleu_score = bleu.score
print(f"Average BLEU: {bleu_score:.4f}")

# BERTScore evaluation
P, R, F1 = bertscore(clarified_texts, original_texts, lang="en")
avg_bertscore_f1 = F1.mean().item()
print(f"Average BERTScore F1: {avg_bertscore_f1:.4f}")

# Save results
sample_df.to_csv("clarified_medical_reports_full_evaluation.csv", index=False)
print("Saved extended evaluations to clarified_medical_reports_full_evaluation.csv")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:0

2025-07-28 16:24:24.063292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753719864.254696      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753719864.322010      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1: 0.9008
Saved extended evaluations to clarified_medical_reports_full_evaluation.csv


In [1]:
# Install required packages
!pip install torch transformers pandas tqdm sentence-transformers rouge-score sacremoses sacrebleu bert-score --quiet

# Imports
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bertscore
import sacrebleu

# Load model and tokenizer
biomedlm_name = "stanford-crfm/BioMedLM"
tokenizer = AutoTokenizer.from_pretrained(biomedlm_name)
tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(biomedlm_name).to(device)
model.eval()

# Load data
df = pd.read_csv("/kaggle/input/precription/health prescription data.csv")
df = df[['SUBJECT_ID', 'ROW_ID', 'HADM_ID', 'CATEGORY', 'ADMISSION_TYPE', 'DIAGNOSIS', 'TEXT']].dropna(subset=['TEXT']).reset_index(drop=True)

# Sample subset
sample_df = df.sample(n=30, random_state=42).reset_index(drop=True)

# Clarification function
def clarify_text(text, max_new_tokens=200):
    prompt = f"Summarize clearly this medical report:\n\n{text}\n\nSummary:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024 - max_new_tokens,
        padding="max_length"
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    clarified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clarified_text.replace(prompt, "").strip()

# Generate summaries
clarified_summaries = []
for report in tqdm(sample_df['TEXT'], desc="Clarifying Medical Reports"):
    try:
        clarified = clarify_text(report)
    except Exception:
        clarified = ""
    clarified_summaries.append(clarified)

sample_df['clarified_summary'] = clarified_summaries

# Initialize evaluation tools
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

rougeL_scores = []
cosine_scores = []
original_texts = sample_df['TEXT'].tolist()
clarified_texts = sample_df['clarified_summary'].tolist()

# Compute embeddings
original_embeddings = embed_model.encode(original_texts, convert_to_tensor=True)
clarified_embeddings = embed_model.encode(clarified_texts, convert_to_tensor=True)

# Compute ROUGE-L and Cosine Similarity
for orig, clar, orig_emb, clar_emb in tqdm(zip(original_texts, clarified_texts, original_embeddings, clarified_embeddings), total=len(original_texts)):
    rougeL_score = rouge.score(orig, clar)['rougeL'].fmeasure
    rougeL_scores.append(rougeL_score)
    cosine_sim = util.cos_sim(orig_emb, clar_emb).item()
    cosine_scores.append(cosine_sim)

avg_rougeL = np.mean(rougeL_scores)
avg_cosine_similarity = np.mean(cosine_scores)
print(f"\nAverage ROUGE-L: {avg_rougeL:.4f}")
print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")

# BLEU evaluation
bleu = sacrebleu.corpus_bleu(clarified_texts, [original_texts])
bleu_score = bleu.score
print(f"Average BLEU: {bleu_score:.4f}")

# BERTScore evaluation
P, R, F1 = bertscore(clarified_texts, original_texts, lang="en")
avg_bertscore_f1 = F1.mean().item()
print(f"Average BERTScore F1: {avg_bertscore_f1:.4f}")

# Show sample output
print(sample_df[['TEXT', 'clarified_summary']].head())

# Save results
sample_df.to_csv("clarified_medical_reports_full_evaluation.csv", index=False)
print("Saved extended evaluations to clarified_medical_reports_full_evaluation.csv")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:0

2025-07-28 16:42:42.331072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753720962.509901      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753720962.561092      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1: 0.9876
                                                TEXT  \
0  Admission Date:  [**2192-5-20**]              ...   
1  Admission Date:  [**2137-12-5**]              ...   
2  Admission Date:  [**2152-10-19**]             ...   
3  Admission Date:  [**2129-5-23**]              ...   
4  Admission Date:  [**2184-6-23**]              ...   

                                   clarified_summary  
0  Summarize clearly this medical report:\n\nAdmi...  
1  Summarize clearly this medical report:\n\nAdmi...  
2  Summarize clearly this medical report:\n\nAdmi...  
3  Summarize clearly this medical report:\n\nAdmi...  
4  Summarize clearly this medical report:\n\nAdmi...  
Saved extended evaluations to clarified_medical_reports_full_evaluation.csv


In [1]:
# Install required packages
!pip install torch transformers pandas tqdm sentence-transformers rouge-score sacremoses sacrebleu bert-score --quiet

import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bertscore
import sacrebleu

# Load MedAlpaca model and tokenizer
model_name = "medalpaca/medalpaca-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
model.eval()

# Load dataset and sample subset
df = pd.read_csv("/kaggle/input/precription/health prescription data.csv")
df = df[['SUBJECT_ID', 'ROW_ID', 'HADM_ID', 'CATEGORY', 'ADMISSION_TYPE', 'DIAGNOSIS', 'TEXT']].dropna(subset=['TEXT']).reset_index(drop=True)
sample_df = df.sample(n=30, random_state=42).reset_index(drop=True)

# Clarification function
def clarify_text_medalpaca(text, max_new_tokens=100):
    prompt = f"### Instruction:\nSummarize the following medical report clearly and concisely.\n\n### Input:\n{text}\n\n### Response:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024 - max_new_tokens,
        padding="max_length"
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    clarified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clarified_text.replace(prompt, "").strip()

# Clarify medical reports
clarified_summaries = []
for report in tqdm(sample_df['TEXT'], desc="Clarifying Medical Reports"):
    try:
        clarified = clarify_text_medalpaca(report)
    except Exception:
        clarified = ""
    clarified_summaries.append(clarified)

sample_df['clarified_summary'] = clarified_summaries

# Initialize evaluation tools
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Compute embeddings
original_texts = sample_df['TEXT'].tolist()
clarified_texts = sample_df['clarified_summary'].tolist()

original_embeddings = embed_model.encode(original_texts, convert_to_tensor=True, batch_size=16)
clarified_embeddings = embed_model.encode(clarified_texts, convert_to_tensor=True, batch_size=16)

rougeL_scores = []
cosine_scores = []

for orig, clar, orig_emb, clar_emb in tqdm(zip(original_texts, clarified_texts, original_embeddings, clarified_embeddings), total=len(original_texts)):
    rougeL_score = rouge.score(orig, clar)['rougeL'].fmeasure
    rougeL_scores.append(rougeL_score)
    cosine_sim = util.cos_sim(orig_emb, clar_emb).item()
    cosine_scores.append(cosine_sim)

avg_rougeL = np.mean(rougeL_scores)
avg_cosine_similarity = np.mean(cosine_scores)

print(f"\nAverage ROUGE-L: {avg_rougeL:.4f}")
print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")

# BLEU evaluation
bleu = sacrebleu.corpus_bleu(clarified_texts, [original_texts])
print(f"Average BLEU: {bleu.score:.4f}")

# BERTScore evaluation
P, R, F1 = bertscore(clarified_texts, original_texts, lang="en")
print(f"Average BERTScore F1: {F1.mean().item():.4f}")

# Show example outputs
print(sample_df[['TEXT', 'clarified_summary']].head())

# Save to CSV
sample_df.to_csv("clarified_medical_reports_medalpaca_evaluation.csv", index=False)
print("Saved evaluations to clarified_medical_reports_medalpaca_evaluation.csv")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.3 MB/s eta 

2025-07-28 18:35:03.656131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753727703.841503      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753727703.897960      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1: 0.9752
                                                TEXT  \
0  Admission Date:  [**2192-5-20**]              ...   
1  Admission Date:  [**2137-12-5**]              ...   
2  Admission Date:  [**2152-10-19**]             ...   
3  Admission Date:  [**2129-5-23**]              ...   
4  Admission Date:  [**2184-6-23**]              ...   

                                   clarified_summary  
0  ### Instruction:\nSummarize the following medi...  
1  ### Instruction:\nSummarize the following medi...  
2  ### Instruction:\nSummarize the following medi...  
3  ### Instruction:\nSummarize the following medi...  
4  ### Instruction:\nSummarize the following medi...  
Saved evaluations to clarified_medical_reports_medalpaca_evaluation.csv


In [1]:
# Install required packages
!pip install torch transformers pandas tqdm sentence-transformers rouge-score sacremoses sacrebleu bert-score --quiet

import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bertscore
import sacrebleu

# Load EleutherAI GPT-Neo model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.resize_token_embeddings(len(tokenizer))
model.eval()

# Load dataset and sample subset
df = pd.read_csv("/kaggle/input/precription/health prescription data.csv")
df = df[['SUBJECT_ID', 'ROW_ID', 'HADM_ID', 'CATEGORY', 'ADMISSION_TYPE', 'DIAGNOSIS', 'TEXT']].dropna(subset=['TEXT']).reset_index(drop=True)
sample_df = df.sample(n=30, random_state=42).reset_index(drop=True)

# Clarification function for GPT-Neo
def clarify_text_gptneo(text, max_new_tokens=100):
    prompt = f"Summarize the following medical report clearly and concisely:\n{text}\nSummary:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024 - max_new_tokens,
        padding="max_length"
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    clarified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clarified_text.replace(prompt, "").strip()

# Clarify medical reports
clarified_summaries = []
for report in tqdm(sample_df['TEXT'], desc="Clarifying Medical Reports"):
    try:
        clarified = clarify_text_gptneo(report)
    except Exception:
        clarified = ""
    clarified_summaries.append(clarified)

sample_df['clarified_summary'] = clarified_summaries

# Evaluation tools
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Compute embeddings
original_texts = sample_df['TEXT'].tolist()
clarified_texts = sample_df['clarified_summary'].tolist()

original_embeddings = embed_model.encode(original_texts, convert_to_tensor=True, batch_size=16)
clarified_embeddings = embed_model.encode(clarified_texts, convert_to_tensor=True, batch_size=16)

rougeL_scores = []
cosine_scores = []

for orig, clar, orig_emb, clar_emb in tqdm(zip(original_texts, clarified_texts, original_embeddings, clarified_embeddings), total=len(original_texts)):
    rougeL_score = rouge.score(orig, clar)['rougeL'].fmeasure
    rougeL_scores.append(rougeL_score)
    cosine_sim = util.cos_sim(orig_emb, clar_emb).item()
    cosine_scores.append(cosine_sim)

avg_rougeL = np.mean(rougeL_scores)
avg_cosine_similarity = np.mean(cosine_scores)

print(f"\nAverage ROUGE-L: {avg_rougeL:.4f}")
print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")

# BLEU evaluation
bleu = sacrebleu.corpus_bleu(clarified_texts, [original_texts])
print(f"Average BLEU: {bleu.score:.4f}")

# BERTScore evaluation
P, R, F1 = bertscore(clarified_texts, original_texts, lang="en")
print(f"Average BERTScore F1: {F1.mean().item():.4f}")

# Show example outputs
print(sample_df[['TEXT', 'clarified_summary']].head())

# Save to CSV
sample_df.to_csv("clarified_medical_reports_gptneo_evaluation.csv", index=False)
print("Saved evaluations to clarified_medical_reports_gptneo_evaluation.csv")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:0

2025-07-29 06:23:40.237037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753770220.395920      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753770220.443712      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1: 0.9827
                                                TEXT  \
0  Admission Date:  [**2192-5-20**]              ...   
1  Admission Date:  [**2137-12-5**]              ...   
2  Admission Date:  [**2152-10-19**]             ...   
3  Admission Date:  [**2129-5-23**]              ...   
4  Admission Date:  [**2184-6-23**]              ...   

                                   clarified_summary  
0  Summarize the following medical report clearly...  
1  Summarize the following medical report clearly...  
2  Summarize the following medical report clearly...  
3  Summarize the following medical report clearly...  
4  Summarize the following medical report clearly...  
Saved evaluations to clarified_medical_reports_gptneo_evaluation.csv


In [1]:
# Install required packages
!pip install torch transformers pandas tqdm sentence-transformers rouge-score sacremoses sacrebleu bert-score --quiet

import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bertscore
import sacrebleu

# Load MedLLaMA2 model and tokenizer
model_name = "llSourcell/medllama2_7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Load dataset and sample 30 entries
df = pd.read_csv("/kaggle/input/precription/health prescription data.csv")
df = df[['SUBJECT_ID', 'ROW_ID', 'HADM_ID', 'CATEGORY', 'ADMISSION_TYPE', 'DIAGNOSIS', 'TEXT']].dropna(subset=['TEXT']).reset_index(drop=True)
sample_df = df.sample(n=30, random_state=42).reset_index(drop=True)

# Clarification function for MedLLaMA2
def clarify_text_medllama(text, max_new_tokens=100):
    prompt = f"### Instruction:\nSummarize the following medical report clearly and concisely.\n\n### Input:\n{text}\n\n### Response:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024 - max_new_tokens,
        padding="max_length"
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    clarified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clarified_text.replace(prompt, "").strip()

# Clarify medical reports
clarified_summaries = []
for report in tqdm(sample_df['TEXT'], desc="Clarifying Medical Reports"):
    try:
        clarified = clarify_text_medllama(report)
    except Exception:
        clarified = ""
    clarified_summaries.append(clarified)

sample_df['clarified_summary'] = clarified_summaries

# Evaluation tools
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
embed_model = SentenceTransformer('all-MiniLM-L6-v2', device=model.device)

# Compute embeddings
original_texts = sample_df['TEXT'].tolist()
clarified_texts = sample_df['clarified_summary'].tolist()

original_embeddings = embed_model.encode(original_texts, convert_to_tensor=True, batch_size=16)
clarified_embeddings = embed_model.encode(clarified_texts, convert_to_tensor=True, batch_size=16)

rougeL_scores = []
cosine_scores = []

for orig, clar, orig_emb, clar_emb in tqdm(zip(original_texts, clarified_texts, original_embeddings, clarified_embeddings), total=len(original_texts)):
    rougeL_score = rouge.score(orig, clar)['rougeL'].fmeasure
    rougeL_scores.append(rougeL_score)
    cosine_sim = util.cos_sim(orig_emb, clar_emb).item()
    cosine_scores.append(cosine_sim)

avg_rougeL = np.mean(rougeL_scores)
avg_cosine_similarity = np.mean(cosine_scores)

print(f"\nAverage ROUGE-L: {avg_rougeL:.4f}")
print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")

# BLEU evaluation
bleu = sacrebleu.corpus_bleu(clarified_texts, [original_texts])
print(f"Average BLEU: {bleu.score:.4f}")

# BERTScore evaluation
P, R, F1 = bertscore(clarified_texts, original_texts, lang="en")
print(f"Average BERTScore F1: {F1.mean().item():.4f}")

# Show example outputs
print(sample_df[['TEXT', 'clarified_summary']].head())

# Save to CSV
sample_df.to_csv("clarified_medical_reports_medllama2_evaluation.csv", index=False)
print("Saved evaluations to clarified_medical_reports_medllama2_evaluation.csv")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 7.0 MB/s eta 0:0

2025-07-29 08:03:58.722906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753776239.058304      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753776239.158088      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1: 0.9752
                                                TEXT  \
0  Admission Date:  [**2192-5-20**]              ...   
1  Admission Date:  [**2137-12-5**]              ...   
2  Admission Date:  [**2152-10-19**]             ...   
3  Admission Date:  [**2129-5-23**]              ...   
4  Admission Date:  [**2184-6-23**]              ...   

                                   clarified_summary  
0  ### Instruction:\nSummarize the following medi...  
1  ### Instruction:\nSummarize the following medi...  
2  ### Instruction:\nSummarize the following medi...  
3  ### Instruction:\nSummarize the following medi...  
4  ### Instruction:\nSummarize the following medi...  
Saved evaluations to clarified_medical_reports_medllama2_evaluation.csv
